In [1]:
# Install dependencies
!pip install music21
!apt install musescore3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  desktop-file-utils fonts-freefont-ttf libevdev2 libgudev-1.0-0 libinput-bin libinput10 libmd4c0
  libmtdev1 libpoppler-qt5-1 libportaudio2 libportmidi0 libqt5core5a libqt5dbus5 libqt5gui5
  libqt5help5 libqt5network5 libqt5printsupport5 libqt5qml5 libqt5qmlmodels5 libqt5qmlworkerscript5
  libqt5quick5 libqt5sql5 libqt5sql5-sqlite libqt5svg5 libqt5widgets5 libqt5xml5 libqt5xmlpatterns5
  libwacom-bin libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0
  musescore-general-soundfont-small musescore3-common qml-module-qt-labs-folderlistmodel
  qml-module-qt-labs-settings qml-module-qtgraphicaleffects qml-module-qtqml
  qml-module-qtqml-models2 qml-module-qtquick-controls qml-module-qtquick-dialogs
  qml-module-qtquick-layouts 

In [2]:
# Import libraries
import os

import numpy as np
import pandas as pd

import copy
import music21
from music21 import converter, chord, note, stream, duration, metadata

In [3]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Change the path if necessary:
root_dir = '/content/drive/Shareddrives/dh401_digital_musicology/asap-dataset-master/Mozart/Piano_Sonatas/8-1/'
os.chdir(root_dir)
!ls

adjusted_midi_score.mid		LEE_J03.mid		    octa.mid
Bogdanovitch01_annotations.txt	left.mid		    path_to_adjusted_durations.mid
Bogdanovitch01.mid		Lo01_annotations.txt	    Rozanski02_annotations.txt
contrast.mid			Lo01.mid		    Rozanski02.mid
Jia01_annotations.txt		midi_score_annotations.txt  sandl.mid
Jia01.mid			midi_score.mid		    test_zero_velocity.mid
LEE_J03_annotations.txt		modified_midi_score.mid     xml_score.musicxml


In [5]:
# Change the file names if necessary:
unperformed_file = root_dir + 'midi_score.mid'
performed_file = root_dir + 'Jia01.mid'

In [6]:
# Load the MIDI file using music21
unperformed_midi = converter.parse(unperformed_file)
performed_midi = converter.parse(performed_file)

# Dynamics

In [7]:
from music21 import articulations
import random

def modulate_left_hand_sequence_chords(midi_stream, staccato_length=0.5, dynamic_range=(60, 90)):
    """
    Modulate left hand sequence chords by applying staccato and random dynamic variations.

    Parameters:
        midi_stream (music21.stream.Score): The original MIDI stream.
        staccato_length (float): Proportion of the original duration to be kept for staccato effect.
        dynamic_range (tuple): The range (min, max) of dynamics to assign randomly to chords.

    Returns:
        music21.stream.Score: The MIDI stream with modified left hand chords.
    """

    new_midi_stream = copy.deepcopy(midi_stream)
    left_hand = new_midi_stream.parts[1]

    # Iterate through all measures in the left hand part
    for measure in left_hand.getElementsByClass(stream.Measure):
        # Check if the entire measure consists of chords
        if all(isinstance(el, chord.Chord) for el in measure.notesAndRests):
            for el in measure.notesAndRests:
                # Apply staccato by reducing the duration
                el.duration.quarterLength *= staccato_length

                # Add a staccato articulation to each note in the chord
                staccato_articulation = articulations.Staccato()
                for note in el.notes:
                    note.articulations.append(staccato_articulation)

                # Randomly choose a dynamic level within the specified range
                dynamic_level = random.randint(*dynamic_range)

                # Apply the dynamic level to all notes in the chord
                for n in el.notes:
                    n.volume.velocity = dynamic_level

    return new_midi_stream

In [8]:
def decrease_left_hand_dynamics(midi_stream, decrease_amount_note, decrease_amount_chord):
    """
    Decrease the dynamics of notes and chords in the left hand part of the MIDI stream,
    with a smaller decrease for notes and a larger decrease for chords.

    Parameters:
        midi_stream (music21.stream.Score): The original MIDI stream.
        decrease_amount_note (int): The amount to decrease the dynamics of notes.
        decrease_amount_chord (int): The amount to decrease the dynamics of notes in chords.

    Returns:
        music21.stream.Score: The modified MIDI stream with decreased dynamics in the left hand.
    """

    # Create a deep copy of the MIDI score to ensure the original is not altered
    new_midi_stream = copy.deepcopy(midi_stream)

    # Access the left hand part; usually the second part in a standard two-hand piano score
    left_hand = new_midi_stream.parts[1]

    # Traverse all elements in the left hand part and decrease their velocity if they are notes or chords
    for el in left_hand.recurse():
        if isinstance(el, note.Note) and el.volume.velocity is not None:
            # Decrease velocity for notes
            el.volume.velocity = max(0, el.volume.velocity - decrease_amount_note)
        elif isinstance(el, chord.Chord):
            # Decrease velocity for each note in the chord
            for single_note in el.notes:
                single_note.volume.velocity = max(0, single_note.volume.velocity - decrease_amount_chord)

    return new_midi_stream

In [9]:
output_modulated_left_hand_sequence_chords = modulate_left_hand_sequence_chords(unperformed_midi)
output_decrease_left_hand_dynamics = decrease_left_hand_dynamics(output_modulated_left_hand_sequence_chords, decrease_amount_note=20, decrease_amount_chord=30)

# Articulations

In [10]:
def add_arpeggios(midi_score, power=2):
    """
    Add arpeggios when there are only chords in a measure.

    Parameters:
        midi_score (music21.stream.Score).
        power (int): The power used to adjust the spacing between arpeggiated notes within a chord.

    Returns:
        music21.stream.Score: The modified MIDI score with arpeggiated chords.
    """
    # Create a deep copy of the MIDI score
    new_midi_score = copy.deepcopy(midi_score)

    # Iterate through each part
    for part in new_midi_score:
      # Skip metadata parts
      if isinstance(part, metadata.Metadata):
        continue

      # Iterate through each measure
      for measure in part:
        # Check if all elements in the measure are chords
        if all(isinstance(chord, music21.chord.Chord) for chord in measure):
          for chord in measure:
            # Iterate through each note
            for n, note in enumerate(chord):
              # Adjust the offset of notes to create the arpeggio effect
              note.offset += n * chord.quarterLength / len(chord.notes)**power

    return new_midi_score

In [11]:
def custom_trills(midi_score):
    """
    Apply custom trills in the MIDI score, by removing some redundant notes.

    Parameters:
        midi_score (music21.stream.Score).

    Returns:
        midi_score (music21.stream.Score).
    """
    # Create a deep copy of the MIDI score
    new_midi_score = copy.deepcopy(midi_score)

    # Iterate through each part
    for part in new_midi_score:
      # Skip metadata parts
      if isinstance(part, metadata.Metadata):
        continue

      # Iterate through each measure
      for measure in part:
        # Iterate through each measure and check if it is a chord of 3 notes
        for chord in measure:
          if isinstance(chord, music21.chord.Chord) and len(chord.pitches) == 3:
            # Check if the first and last notes are the same
            if chord.pitches[0] == chord.pitches[2]:
              # Check if the letter names of the notes are next to each other
              if (ord(chord.pitches[1].name[0]) == ord(chord.pitches[0].name[0]) + 1) or (ord(chord.pitches[1].name[0]) == ord(chord.pitches[0].name[0]) - 1):
                # Remove redundant note in the chord
                chord = chord.removeRedundantPitchClasses(inPlace=True)

    return new_midi_score

In [12]:
output_add_arpeggios = add_arpeggios(output_decrease_left_hand_dynamics, 2)
output_custom_trills = custom_trills(output_add_arpeggios)

In [13]:
print(output_custom_trills)

<music21.stream.Score 0x7d8662411330>


# Phrases

In [14]:
from music21 import converter, stream, midi, tempo

def adjust_velocity(midi_file_path):
    """
    Change the velocity of musical phrases so they sound more performed.

    Parameters:
        midi_file_path (string)

    Returns:
        music21.stream.Score: The modified MIDI score with phrases sounding more natural.
    """


    # Load the MIDI file and create a copy to work on
    ## original_score = converter.parse(midi_file_path)
    modified_score = copy.deepcopy(midi_file_path).flatten()

    last_velocity = None
    current_phrase = []

    for element in modified_score.recurse():

        if isinstance(element, tempo.MetronomeMark):
            element.number = 130


        # Only modify Note objects
        if 'Note' in element.classes or 'Chord' in element.classes:
            if last_velocity is None or element.volume.velocity == last_velocity:
                current_phrase.append(element)
            else:
                # Adjust velocities at the end and beginning of each phrase
                adjust_phrase_velocities(current_phrase)
                current_phrase = [element]
            last_velocity = element.volume.velocity


    # Adjust the last phrase
      # adjust_phrase_velocities(current_phrase)

      # for i in range(len(modified_score.notes) - 1):  # Exclude the last note as it has no next note

      #         current_note = modified_score.notes[i]
      #         next_note = modified_score.notes[i + 1]
      #         current_note.duration.quarterLength *= next_note.volume.velocity/64

    # Return the modified MIDI file
    return modified_score



def adjust_phrase_velocities(phrase):
    length = len(phrase)
    factor = 15  # Define how much to change the velocity

    if length > 2:
      for i in range(length):
        wave_peak = length // 2
        if i <= wave_peak:
          new_velocity = int(phrase[i].volume.velocity + factor * (i / wave_peak))

        else:
          new_velocity = int(phrase[i].volume.velocity + factor * ((length - i) / (length - wave_peak)))

        phrase[i].volume.velocity = new_velocity

In [15]:
output_adjust_velocity = adjust_velocity(output_custom_trills)

In [16]:
output_adjust_velocity.write('midi', '/content/drive/Shareddrives/dh401_digital_musicology/assignment2/output_adjust_velocity.mid')

'/content/drive/Shareddrives/dh401_digital_musicology/assignment2/output_adjust_velocity.mid'

# IOI

In [17]:
def check_for_contrast(chord, previous_chord, x, ratio, m):
    if previous_chord == None:
      previous_chord = chord
      return

    # Else, if both previous and actual event are sounded, calculate the ratio and act accordingly
    if previous_chord.duration.quarterLength/chord.duration.quarterLength == ratio:
      chord.offset = chord.offset - x
      print('happened! measure n°', m, '||', previous_chord.offset+1, chord.offset+1+x, previous_chord)

In [18]:
from music21 import stream, note, chord

def decrease_ioi_contrast(midi_score, ratio, x):
    new_midi_score = stream.Score()

    for part in midi_score:
        if not isinstance(part, stream.Stream):
            new_midi_score.append(part)
            continue

        new_part = stream.Part()
        part_flattened = part.flatten()

        happened = False
        previous_chord = None
        for event in part_flattened:
            chord = copy.deepcopy(event)
            chord.offset = event.offset
            new_part.append(chord)
            if not isinstance(chord, (note.Note, chord.Chord)):
                previous_chord = None
                continue

            if previous_chord is None:
                previous_chord = chord
                continue

            if happened:
                happened = False

            if chord.duration.quarterLength != 0:
                if previous_chord.quarterLength / chord.quarterLength == ratio:
                    new_part[-1].offset = new_part[-1].offset - x
                    happened = True

            previous_chord = chord

        new_midi_score.append(new_part)

    return new_midi_score

In [19]:
final_midi_output_2 = decrease_ioi_contrast(output_adjust_velocity, 2/1, 1/4)

In [20]:
final_midi_output_2.write('midi', '/content/drive/Shareddrives/dh401_digital_musicology/assignment2/final_midi_output_2.mid')

'/content/drive/Shareddrives/dh401_digital_musicology/assignment2/final_midi_output_2.mid'